In [27]:
# setwd("home/bogdan/Desktop/Amgen_code")
library(ggplot2)
library(reshape2)
library(dplyr)
library(tidyr)
library(car)
library(lmtest)
library(nortest)
library(multcomp)
library(emmeans)
# library(tidyverse)

In [2]:
x = read.delim("input.txt", header = T, sep="\t", stringsAsFactors = FALSE)
colnames(x)

[1] "subject"         "timepoint"       "analyte_value"   "marker"         
[5] "treatment_group"

In [3]:
# DATA EXPLORATION, in concordance with the longitudinal treatment evaluation

In [4]:
print("the subjects enrolled in the study :")
table(x$subject)
print("the number of subjects is :")
length(unique(x$subject))
print("the timepoints :")
table(x$timepoint)
print("the markers :")
table(x$marker)
print("the treatment groups :")
table(x$treatment_group)

[1] "the subjects enrolled in the study :"



 A  B  C  D  E  F  G  H  J  K  L  M  N  P  Q  R  S  T  U  V  W 
12 15 15 15 15  9 15  3  9 15  5 12 15 15 12 12 15 12 15  6 12 

[1] "the number of subjects is :"


[1] 21

[1] "the timepoints :"



 DAY1 DAY15 DAY22 DAY29  DAY8 
   56    51    48    45    54 

[1] "the markers :"



C4 C8 TG 
85 85 84 

[1] "the treatment groups :"



TA TB TC 
83 87 84 

In [5]:
# coding subject, timepoint, marker, treatment_group as FACTORS
x$subject <- factor(x$subject)
x$timepoint <- factor(x$timepoint)
x$marker <- factor(x$marker)
x$treatment_group <- factor(x$treatment_group)

# ensuring that 'timepoint' is a factor with the desired order
x$timepoint <- factor(x$timepoint, levels = c("DAY1", "DAY8", "DAY15", "DAY22", "DAY29"))

In [6]:
head(x,2)
tail(x,2)

,subject,timepoint,analyte_value,marker,treatment_group
,<fct>,<fct>,<int>,<fct>,<fct>
1,A,DAY1,169,C4,TA
2,A,DAY1,207,C8,TA


,subject,timepoint,analyte_value,marker,treatment_group
,<fct>,<fct>,<int>,<fct>,<fct>
253,U,DAY8,225,C8,TC
254,U,DAY8,76,TG,TC


In [7]:
# 5. Please fit an approriate statistical model for each marker with analyte_value as dependent variables, 
# and timepoint and treatment as independent variables (both are conisidered factors). 
# Assume subjects are randomly drawn from a population. 
# With each model, please report the significance of treatment effect and the contrast between Day 22 and Day 8.

In [8]:
# Given the statistical analysis shown in the previous part (part4), 
# focused on the gaussian distribution and  homescedasticity / heteroscedasticity of the data,
# I believe that we could use simple linear models. 
# Given a few exceptions in the gaussian distribution and homoscedasticity of the analyte values, 
# that we have observed, we may also use GLM.

In [9]:
# In our models, we do NOT consider INTERACTIONS between TIMEPOINT and TREATMENT.

In [10]:
# Separating the data frame per marker

x$timepoint = factor(x$timepoint, levels = c("DAY1", "DAY8", "DAY15", "DAY22", "DAY29"))
x$treatment_group = factor(x$treatment_group)

# Set reference levels
x$treatment_group <- relevel(x$treatment_group, ref = "TA")
x$timepoint <- relevel(x$timepoint, ref = "DAY1")

x_C4 = split(x, x$marker)$C4
x_C8 = split(x, x$marker)$C8
x_TG = split(x, x$marker)$TG

In [11]:
df = x_C4

# Fit Generalized Linear Model
# glm_model <- glm(analyte_value ~ timepoint + treatment_group, 
#                  family = gaussian(), # the model assumes normally distributed residuals.
#                  data = df)
# Get the model summary
# glm_summary <- summary(glm_model)
# print(glm_summary)

# Fit a Linear Model
lm_model <- lm(analyte_value ~ timepoint + treatment_group, data = df)

# Get the model summary
lm_summary <- summary(lm_model)

# Print the summary to examine significance of effects
print(lm_summary)

# Extracting the contrast of interest : 
df$timepoint <- factor(df$timepoint, levels = c("DAY1", "DAY8", "DAY15", "DAY22", "DAY29"))

# Compute estimated marginal means
emm <- emmeans(lm_model, ~ timepoint)

# Specify the contrast between Day 22 and Day 8
contrast_results <- contrast(emm, list("Day22 vs Day8" = c(0, -1, 0, 1, 0)))

# Print the results
print(contrast_results)


Call:
lm(formula = analyte_value ~ timepoint + treatment_group, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-755.9 -330.9 -147.1  219.2 1846.1 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)         562.19     152.06   3.697 0.000404 ***
timepointDAY8       320.46     184.53   1.737 0.086393 .  
timepointDAY15       76.74     186.55   0.411 0.681916    
timepointDAY22       20.37     189.96   0.107 0.914899    
timepointDAY29     -119.94     192.79  -0.622 0.535672    
treatment_groupTB   127.29     148.39   0.858 0.393637    
treatment_groupTC  -236.43     150.10  -1.575 0.119277    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 557.6 on 78 degrees of freedom
Multiple R-squared:  0.1444,	Adjusted R-squared:  0.07853 
F-statistic: 2.193 on 6 and 78 DF,  p-value: 0.05236

 contrast      estimate  SE df t.ratio p.value
 Day22 vs Day8     -300 193 78  -1.556  0.1237

Results are averaged 

In [12]:
# INTERPRETATION :

# REFERENCE LEVELS : 
# When all predictors are at their reference levels (treatment "TA", timepoint "DAY1"), 
# the estimated average analyte_value is statistically significant, indicating a strong baseline effect.

# For the timepoint "DAY8" compared to the reference timepoint, the analyte_value is marginally significant 
# (p-value just above 0.05), suggesting a trend toward significance.

# Additionally, Multiple R-squared and Adjusted R-squared are very low, indicating that the model does not explain 
# much more variance than would be expected by chance. 
# There is a relatively weak relationship between the predictors and the outcome.

# Only the intercept is statistically significant.
# The F-statistic is close to 0.05, indicating that the model is marginally significant as a whole 
# but doesn’t strongly explain the variability in the response variable.

# Contrast DAY22 - DAY8 : the difference is not statistically significant.

In [13]:
df = x_C8

# Fit Generalized Linear Model
# glm_model <- glm(analyte_value ~ timepoint + treatment_group, 
#                  family = gaussian(), # the model assumes normally distributed residuals.
#                  data = df)
# Get the model summary
# glm_summary <- summary(glm_model)
# print(glm_summary)

# Fit a Linear Model
lm_model <- lm(analyte_value ~ timepoint + treatment_group, data = df)

# Get the model summary
lm_summary <- summary(lm_model)

# Print the summary to examine significance of effects
print(lm_summary)

# Extracting the contrast of interest : 
df$timepoint <- factor(df$timepoint, levels = c("DAY1", "DAY8", "DAY15", "DAY22", "DAY29"))

# Compute estimated marginal means
emm <- emmeans(lm_model, ~ timepoint)

# Specify the contrast between Day 22 and Day 8
contrast_results <- contrast(emm, list("Day22 vs Day8" = c(0, -1, 0, 1, 0)))

# Print the results
print(contrast_results)


Call:
lm(formula = analyte_value ~ timepoint + treatment_group, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-1848.8  -589.3  -172.8   177.7  4731.4 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)   
(Intercept)         616.15     357.07   1.726  0.08839 . 
timepointDAY8       449.18     433.31   1.037  0.30311   
timepointDAY15      -23.38     438.06  -0.053  0.95757   
timepointDAY22      -67.02     446.07  -0.150  0.88095   
timepointDAY29     -323.47     452.71  -0.715  0.47704   
treatment_groupTB  1108.49     348.45   3.181  0.00211 **
treatment_groupTC   -36.32     352.48  -0.103  0.91819   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1309 on 78 degrees of freedom
Multiple R-squared:  0.1854,	Adjusted R-squared:  0.1227 
F-statistic: 2.959 on 6 and 78 DF,  p-value: 0.01178

 contrast      estimate  SE df t.ratio p.value
 Day22 vs Day8     -516 453 78  -1.140  0.2578

Results are averaged 

In [14]:
# INTERPRETATION : 

# The estimated baseline analyte_value when timepoint and treatment_group are at their reference levels is marginally significant.
# The p-value is slightly above the conventional 0.05 threshold, 
# suggesting that the baseline value is not significantly different from zero, but there is a hint of a difference.

# TREATMENT TB : The analyte_value for the treatment group "TB" is estimated to be 1108 units higher compared 
# to the reference treatment group. 
# This effect is statistically significant (p-value < 0.01), suggesting a strong difference in analyte_value for the "TB" treatment group.

# Additionally, Multiple R-squared and Adjusted R-squared are very low, indicating that the model does not explain 
# much more variance than would be expected by chance. 
# There is a relatively weak relationship between the predictors and the outcome.

# Only treatment_groupTB shows a statistically significant effect on analyte_value,
# suggesting a notable impact of this treatment group.

# The F-statistic is significant (p-value < 0.05), indicating that the model as a whole is statistically significant 
# and that at least one of the predictors contributes to explaining the variability in analyte_value.

# Contrast DAY22 - DAY8 : the difference is not statistically significant.

In [15]:
df = x_TG

# Fit Generalized Linear Model
# glm_model <- glm(analyte_value ~ timepoint + treatment_group, 
#                  family = gaussian(), # the model assumes normally distributed residuals.
#                  data = df)
# Get the model summary
# glm_summary <- summary(glm_model)
# print(glm_summary)

# Fit a Linear Model
lm_model <- lm(analyte_value ~ timepoint + treatment_group, data = df)

# Get the model summary
lm_summary <- summary(lm_model)

# Print the summary to examine significance of effects
print(lm_summary)

# Extracting the contrast of interest : 
df$timepoint <- factor(df$timepoint, levels = c("DAY1", "DAY8", "DAY15", "DAY22", "DAY29"))

# Compute estimated marginal means
emm <- emmeans(lm_model, ~ timepoint)

# Specify the contrast between Day 22 and Day 8
contrast_results <- contrast(emm, list("Day22 vs Day8" = c(0, -1, 0, 1, 0)))

# Print the results
print(contrast_results)


Call:
lm(formula = analyte_value ~ timepoint + treatment_group, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-178.57  -62.16  -20.83   36.88  493.43 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)         42.165     33.471   1.260 0.211567    
timepointDAY8      157.407     39.634   3.972 0.000159 ***
timepointDAY15      76.890     39.986   1.923 0.058189 .  
timepointDAY22      94.455     40.672   2.322 0.022857 *  
timepointDAY29      32.887     41.317   0.796 0.428506    
treatment_groupTB   -5.556     31.669  -0.175 0.861187    
treatment_groupTC  -22.598     32.097  -0.704 0.483522    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 118.1 on 77 degrees of freedom
Multiple R-squared:  0.2021,	Adjusted R-squared:  0.1399 
F-statistic:  3.25 on 6 and 77 DF,  p-value: 0.006689

 contrast      estimate   SE df t.ratio p.value
 Day22 vs Day8      -63 40.8 77  -1.541  0.1273

Results a

In [16]:
# INTERPRETATION :

# DAY 8 : The analyte_value is significantly higher on DAY 8 compared to the reference timepoint, 
# and this effect is highly statistically significant.

# DAY 22: The positive estimate indicates a meaningful increase in analyte_value, 
# and the p-value below 0.05 confirms that this effect is statistically significant.

# Additionally, Multiple R-squared and Adjusted R-squared are very low, indicating that the model does not explain 
# much more variance than would be expected by chance. 
# There is a relatively weak relationship between the predictors and the outcome.

# DAY8 and DAY22 have a significant impact on analyte_value, with DAY8 showing the strongest effect. 
# DAY15 is marginally significant, while DAY29 does not significantly impact analyte_value.

# The p-value of the F-statistics indicates that the overall model is statistically significant. 

# Contrast DAY22 - DAY8 : the difference is not statistically significant.

In [17]:
# 6. If you had to analyze 1 million markers, how would you parallelize the tests from question 4 
# on a multi-processor machine? Please demonstrate this parallelization using just the markers 
# available in the spreadsheet.

In [18]:
# Let's consider the code that we have used before, when considering the treatment TB. 

x_wide <- reshape(
  x,
  idvar = c("subject", "marker", "treatment_group"),  # Variables to keep constant
  timevar = "timepoint",                              # Variable that will become columns
  direction = "wide",                                 # Convert from long to wide format
  v.names = "analyte_value"                           # Values to spread across the new columns
)

a <- x_wide %>% dplyr::select(subject, 
                        marker,
                        treatment_group,
                        analyte_value.DAY1,
                        analyte_value.DAY8)


a_tb <- a %>% filter(treatment_group == "TB")
head(a_tb, 2)
tail(a_tb, 2)

,subject,marker,treatment_group,analyte_value.DAY1,analyte_value.DAY8
,<fct>,<fct>,<fct>,<int>,<int>
1,C,C4,TB,2328,2534
2,C,C8,TB,6456,6384


,subject,marker,treatment_group,analyte_value.DAY1,analyte_value.DAY8
,<fct>,<fct>,<fct>,<int>,<int>
20,H,C8,TB,NA,1185
21,H,TG,TB,NA,645


In [19]:
df = a_tb 

# Previously, when answering the question # 4, we have written the following piece of code 
# We used T.test with var.equal = TRUE and Welch T.test.
# We demonstrate the code by using Welch T.test segment of code.

results <- df %>%
  group_by(marker) %>%
  summarise(
    t_test = list(t.test(analyte_value.DAY8, analyte_value.DAY1)),
    .groups = 'drop'
  ) %>%
  mutate(
    p_value = sapply(t_test, function(x) x$p.value),
    significance_label = ifelse(p_value < 0.05, "Significant", "Not Significant")
  )
# Print the results
print("Differences in the Treatment TB :")
print("use Welsch T.test:")
print(results)

[1] "Differences in the Treatment TB :"
[1] "use Welsch T.test:"
# A tibble: 3 × 4
  marker t_test  p_value significance_label
  <fct>  <list>    <dbl> <chr>             
1 C4     <htest>  0.469  Not Significant   
2 C8     <htest>  0.675  Not Significant   
3 TG     <htest>  0.0682 Not Significant   


In [20]:
# In order to parallelize, we use the library "parallel" :
library(parallel)

# Number of cores to use
num_cores <- detectCores() - 1
print("the number of cores is :") 
print(num_cores)


[1] "the number of cores is :"
[1] 7


In [21]:
# Define the function to perform Welch T-test for each marker

compute_t_test <- function(marker_data) {
  
  t_test_result <- t.test(marker_data$analyte_value.DAY8, marker_data$analyte_value.DAY1)
  
  p_value <- t_test_result$p.value
  significance_label <- ifelse(p_value < 0.05, "Significant", "Not Significant")
  
  return(list(p_value = p_value, significance_label = significance_label))
}

# Split data by marker
df = a_tb 
marker_list <- split(df, df$marker)

# Initialize parallel computing, perform parallel computation, and stop the cluster
cl <- makeCluster(num_cores)
results_list <- parLapply(cl, marker_list, compute_t_test)
stopCluster(cl)


In [22]:
print("the results produced by the R cluster :")
results_list

# we place these results into a dataframe 
# results_parallel_processing <- data.frame(
#  marker = c(names(results_list[1]), names(results_list[2]), names(results_list[3])),
#  p_value = c(results_list[[1]]$p_value, results_list[[2]]$p_value, results_list[[3]]$p_value),
#  significance_label = c(results_list[[1]]$significance_label, results_list[[2]]$significance_label, results_list[[3]]$significance_label) 
# )

# print(results_parallel_processing)

# we generalize to any number of markers :

results_parallel_processing.df<- data.frame(
  marker = names(results_list),
  p_value = sapply(results_list, function(x) x$p_value),
  significance_label = sapply(results_list, function(x) x$significance_label)
)

print("Differences in the Treatment TB between DAY 1 and DAY 8:")
print("use Welsch T.test:")
print(results_parallel_processing.df)

[1] "the results produced by the R cluster :"


$C4
$C4$p_value
[1] 0.4693234

$C4$significance_label
[1] "Not Significant"


$C8
$C8$p_value
[1] 0.6745258

$C8$significance_label
[1] "Not Significant"


$TG
$TG$p_value
[1] 0.0682489

$TG$significance_label
[1] "Not Significant"

[1] "Differences in the Treatment TB between DAY 1 and DAY 8:"
[1] "use Welsch T.test:"
   marker   p_value significance_label
C4     C4 0.4693234    Not Significant
C8     C8 0.6745258    Not Significant
TG     TG 0.0682489    Not Significant


In [23]:
# 7. Please automate the table view for different layouts, i.e., write a function that takes the input data 
# and another parameter indicating a categorical (nominal) variable so that the output of this function 
# will produce a new table in which each level of the indicated variable become a separate column filled 
# with corresponding analyte_value and others columns remain. 
# In the data provided for the quiz, except "analyte_value", all other variables are factors :)

In [24]:
# we use the manin data frame x :

head(x,2)
tail(x,2)

,subject,timepoint,analyte_value,marker,treatment_group
,<fct>,<fct>,<int>,<fct>,<fct>
1,A,DAY1,169,C4,TA
2,A,DAY1,207,C8,TA


,subject,timepoint,analyte_value,marker,treatment_group
,<fct>,<fct>,<int>,<fct>,<fct>
253,U,DAY8,225,C8,TC
254,U,DAY8,76,TG,TC


In [25]:
table_view <- function(data, categorical_variable) {
  
    # Reshape the data according to the instructions :
    reshaped_data <- data %>%
                   pivot_wider(names_from = categorical_variable, values_from = analyte_value)
  
    return(reshaped_data)

}

In [26]:
# Categorical variable : "marker" ; print the reshaped data

reshaped_df <- table_view(x, "marker")
head(as.data.frame(reshaped_df), 6)

# Categorical variable : "treatment_group" ; print the reshaped data

reshaped_df <- table_view(x, "treatment_group")
head(as.data.frame(reshaped_df), 6)

# Categorical variable : "timepoint" ; print the reshaped data

reshaped_df <- table_view(x, "timepoint")
head(as.data.frame(reshaped_df), 6)

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(categorical_variable)

  # Now:
  data %>% select(all_of(categorical_variable))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


,subject,timepoint,treatment_group,C4,C8,TG
,<fct>,<fct>,<fct>,<int>,<int>,<int>
1,A,DAY1,TA,169,207,10
2,B,DAY1,TA,636,821,19
3,F,DAY1,TA,254,1213,6
4,L,DAY1,TA,90,96,NA
5,M,DAY1,TA,1377,1139,42
6,N,DAY1,TA,319,117,20


,subject,timepoint,marker,TA,TB,TC
,<fct>,<fct>,<fct>,<int>,<int>,<int>
1,A,DAY1,C4,169,NA,NA
2,A,DAY1,C8,207,NA,NA
3,A,DAY1,TG,10,NA,NA
4,B,DAY1,C4,636,NA,NA
5,B,DAY1,C8,821,NA,NA
6,B,DAY1,TG,19,NA,NA


,subject,marker,treatment_group,DAY1,DAY15,DAY22,DAY29,DAY8
,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>
1,A,C4,TA,169,308,NA,290,187
2,A,C8,TA,207,418,NA,479,156
3,A,TG,TA,10,68,NA,118,21
4,B,C4,TA,636,1151,640,118,1155
5,B,C8,TA,821,1021,574,159,1035
6,B,TG,TA,19,231,137,25,693
